In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# PyOD imports - comprehensive set
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.abod import ABOD
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
from pyod.models.loda import LODA
from pyod.models.lscp import LSCP
from pyod.models.so_gaal import SO_GAAL
from pyod.models.vae import VAE
from pyod.models.auto_encoder import AutoEncoder

# For ensemble and evaluation
from pyod.models.combination import average, maximization, median
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import joblib
import os
import time
from tqdm.notebook import tqdm

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("=" * 80)
print("ADVANCED UNSUPERVISED ANOMALY DETECTION PIPELINE")
print("=" * 80)

# =====================================================================
# STEP 1: LOAD AND PREPARE DATA
# =====================================================================
print("\n[STEP 1] Loading preprocessed data...")

# Load the normalized dataset
data_path = '../../results/benign_preprocessed/benign_v5_normalized.csv'
df = pd.read_csv(data_path)
print(f"✓ Loaded normalized dataset: {df.shape}")

# CRITICAL: Filter out contaminated high-rate traffic
# Based on our analysis, rates > 50,000 pps are suspicious for IoT
print("\n[DATA CLEANING] Checking for contaminated samples...")
original_size = len(df)

# Since we're using normalized data, we need a different approach
# Check if we have a log_rate column (indicates preprocessing was done)
if 'log_rate' in df.columns:
    # Use statistical threshold instead
    print("  - Using statistical outlier removal on normalized data")
    # Remove extreme outliers (> 3 std deviations)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    z_scores = np.abs(stats.zscore(df[numeric_cols]))
    outlier_mask = (z_scores > 3).any(axis=1)
    df_clean = df[~outlier_mask]
    print(f"  ✓ Removed {outlier_mask.sum()} extreme outliers based on z-score > 3")
else:
    # If we have the original Rate column
    df_clean = df
    print("  - No filtering applied (working with preprocessed data)")

print(f"✓ Clean dataset: {df_clean.shape}")

# Select features for anomaly detection
# Exclude categorical and identifier columns
exclude_cols = ['Protocol Type', 'size_category', 'Header_Length', 'Time_To_Live']
feature_cols = [col for col in df_clean.columns if col not in exclude_cols]
X = df_clean[feature_cols].values

# CRITICAL: Handle inf/nan values
print("\n[DATA VALIDATION] Checking for inf/nan values...")
n_inf = np.isinf(X).sum()
n_nan = np.isnan(X).sum()
print(f"  - Infinite values found: {n_inf}")
print(f"  - NaN values found: {n_nan}")

if n_inf > 0 or n_nan > 0:
    print("  - Replacing inf values with max finite values...")
    # Replace inf with max finite values
    X = np.nan_to_num(X, nan=0.0, posinf=np.finfo(np.float32).max, 
                      neginf=np.finfo(np.float32).min)
    
    # Additional check for very large values
    max_val = np.abs(X).max()
    if max_val > 1e10:
        print(f"  - Found very large values (max: {max_val:.2e})")
        print("  - Clipping to reasonable range...")
        X = np.clip(X, -1e6, 1e6)
    
    print("  ✓ Data cleaned and validated")

# Convert to float32 for memory efficiency
X = X.astype(np.float32)
print(f"✓ Features for anomaly detection: {X.shape[1]}")

# Initialize results directory
results_dir = '../../results/pyod_advanced/'
os.makedirs(results_dir, exist_ok=True)

# =====================================================================
# STEP 2: CLASSICAL ANOMALY DETECTION MODELS
# =====================================================================
print("\n[STEP 2] Training Classical Anomaly Detection Models...")

# Define contamination rate based on our cleaned data
# Since we removed obvious outliers, use a lower contamination rate
contamination = 0.02  # 2% expected anomalies in cleaned data

# Initialize models with optimized parameters
classical_models = {
    'IForest': IForest(
        contamination=contamination,
        n_estimators=200,
        max_features=0.8,
        bootstrap=True,
        random_state=42
    ),
    'LOF': LOF(
        contamination=contamination,
        n_neighbors=30,
        algorithm='auto',
        n_jobs=-1
    ),
    'CBLOF': CBLOF(
        contamination=contamination,
        n_clusters=8,
        alpha=0.9,
        random_state=42
    ),
    'KNN': KNN(
        contamination=contamination,
        n_neighbors=10,
        method='mean',
        n_jobs=-1
    ),
    'HBOS': HBOS(
        contamination=contamination,
        n_bins=50,
        alpha=0.1
    ),
    'COPOD': COPOD(
        contamination=contamination
    ),
    'ECOD': ECOD(
        contamination=contamination,
        n_jobs=-1
    ),
    'LODA': LODA(
        contamination=contamination,
        n_bins=10,
        n_random_cuts=100
    )
}

# Train models and collect results
classical_results = {}
classical_scores = {}

print("\nTraining classical models...")
for name, model in tqdm(classical_models.items(), desc="Classical Models", 
                       total=len(classical_models)):
    print(f"\n  Training {name}...")
    start_time = time.time()
    
    try:
        # Fit the model
        model.fit(X)
        
        # Get anomaly scores and predictions
        scores = model.decision_scores_
        predictions = model.predict(X)
        
        # Calculate statistics
        n_outliers = np.sum(predictions == 1)
        train_time = time.time() - start_time
        
        classical_results[name] = {
            'model': model,
            'scores': scores,
            'predictions': predictions,
            'n_outliers': n_outliers,
            'outlier_percentage': n_outliers / len(X) * 100,
            'train_time': train_time,
            'score_mean': np.mean(scores),
            'score_std': np.std(scores),
            'score_threshold': model.threshold_
        }
        classical_scores[name] = scores
        
        print(f"    ✓ Completed in {train_time:.2f}s")
        print(f"    ✓ Outliers detected: {n_outliers} ({n_outliers/len(X)*100:.2f}%)")
        
    except Exception as e:
        print(f"    ✗ Error: {str(e)}")
        classical_results[name] = {'error': str(e)}

# =====================================================================
# STEP 3: DEEP LEARNING ANOMALY DETECTION
# =====================================================================
print("\n[STEP 3] Training Deep Learning Models...")

# Prepare data for deep learning models
# Use fewer features for neural networks to avoid overfitting
print("  - Applying PCA for dimensionality reduction...")
pca_reducer = PCA(n_components=0.95, random_state=42)
X_reduced = pca_reducer.fit_transform(X)
print(f"  ✓ Reduced dimensions for DL: {X.shape[1]} -> {X_reduced.shape[1]}")

# Deep learning models
dl_models = {
    'AutoEncoder': AutoEncoder(
        hidden_neurons=[X_reduced.shape[1], 32, 16, 16, 32, X_reduced.shape[1]],
        contamination=contamination,
        epochs=50,
        batch_size=64,
        dropout_rate=0.2,
        l2_regularizer=0.1,
        validation_size=0.1,
        verbose=0,
        random_state=42
    ),
    'VAE': VAE(
        encoder_neurons=[X_reduced.shape[1], 64, 32],
        decoder_neurons=[32, 64, X_reduced.shape[1]],
        contamination=contamination,
        epochs=30,
        batch_size=64,
        dropout_rate=0.2,
        l2_regularizer=0.1,
        validation_size=0.1,
        verbose=0,
        random_state=42
    )
}

dl_results = {}
dl_scores = {}

print("\nTraining deep learning models...")
for name, model in tqdm(dl_models.items(), desc="Deep Learning Models", 
                       total=len(dl_models)):
    print(f"\n  Training {name}...")
    start_time = time.time()
    
    try:
        # Fit the model
        model.fit(X_reduced)
        
        # Get anomaly scores and predictions
        scores = model.decision_scores_
        predictions = model.predict(X_reduced)
        
        # Calculate statistics
        n_outliers = np.sum(predictions == 1)
        train_time = time.time() - start_time
        
        dl_results[name] = {
            'model': model,
            'scores': scores,
            'predictions': predictions,
            'n_outliers': n_outliers,
            'outlier_percentage': n_outliers / len(X) * 100,
            'train_time': train_time,
            'score_mean': np.mean(scores),
            'score_std': np.std(scores),
            'score_threshold': model.threshold_
        }
        dl_scores[name] = scores
        
        print(f"    ✓ Completed in {train_time:.2f}s")
        print(f"    ✓ Outliers detected: {n_outliers} ({n_outliers/len(X)*100:.2f}%)")
        
    except Exception as e:
        print(f"    ✗ Error: {str(e)}")
        dl_results[name] = {'error': str(e)}

# Combine all results
all_results = {**classical_results, **dl_results}
all_scores = {**classical_scores, **dl_scores}

# =====================================================================
# STEP 4: ENSEMBLE METHODS
# =====================================================================
print("\n[STEP 4] Creating Ensemble Models...")

# Collect successful model scores
successful_scores = []
successful_models = []
for name, scores in all_scores.items():
    if isinstance(scores, np.ndarray) and len(scores) == len(X):
        successful_scores.append(scores.reshape(-1, 1))
        successful_models.append(name)

if len(successful_scores) >= 3:
    # Combine scores
    scores_array = np.hstack(successful_scores)
    
    # Create ensemble scores
    ensemble_avg = average(scores_array)
    ensemble_max = maximization(scores_array)
    ensemble_median = median(scores_array)
    
    # Determine thresholds
    ensemble_results = {
        'Average': {
            'scores': ensemble_avg,
            'threshold': np.percentile(ensemble_avg, 100 - contamination * 100),
            'models_used': successful_models
        },
        'Maximum': {
            'scores': ensemble_max,
            'threshold': np.percentile(ensemble_max, 100 - contamination * 100),
            'models_used': successful_models
        },
        'Median': {
            'scores': ensemble_median,
            'threshold': np.percentile(ensemble_median, 100 - contamination * 100),
            'models_used': successful_models
        }
    }
    
    # Add predictions
    for name, data in ensemble_results.items():
        data['predictions'] = (data['scores'] > data['threshold']).astype(int)
        data['n_outliers'] = np.sum(data['predictions'])
        print(f"✓ Ensemble {name}: {data['n_outliers']} outliers ({data['n_outliers']/len(X)*100:.2f}%)")

# =====================================================================
# STEP 5: VISUALIZATION AND ANALYSIS
# =====================================================================
print("\n[STEP 5] Creating Visualizations...")

# 1. Model Performance Comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Training Time Comparison
ax1 = axes[0, 0]
model_names = [name for name, result in all_results.items() if 'train_time' in result]
train_times = [result['train_time'] for name, result in all_results.items() if 'train_time' in result]
colors = ['lightblue' if name in classical_models else 'lightcoral' for name in model_names]

bars = ax1.bar(range(len(model_names)), train_times, color=colors)
ax1.set_xticks(range(len(model_names)))
ax1.set_xticklabels(model_names, rotation=45, ha='right')
ax1.set_ylabel('Training Time (seconds)')
ax1.set_title('Model Training Time Comparison')
ax1.legend(['Classical', 'Deep Learning'], loc='upper right')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}s', ha='center', va='bottom')

# Plot 2: Outlier Detection Rates
ax2 = axes[0, 1]
outlier_rates = [result['outlier_percentage'] for name, result in all_results.items() 
                 if 'outlier_percentage' in result]
model_names_outliers = [name for name, result in all_results.items() 
                       if 'outlier_percentage' in result]

bars = ax2.bar(range(len(model_names_outliers)), outlier_rates, color='skyblue')
ax2.axhline(y=contamination*100, color='red', linestyle='--', 
            label=f'Expected ({contamination*100}%)')
ax2.set_xticks(range(len(model_names_outliers)))
ax2.set_xticklabels(model_names_outliers, rotation=45, ha='right')
ax2.set_ylabel('Outlier Detection Rate (%)')
ax2.set_title('Outlier Detection Rates by Model')
ax2.legend()

# Plot 3: Score Distribution Comparison (top 5 models)
ax3 = axes[1, 0]
top_models = list(all_scores.keys())[:5]
for i, model_name in enumerate(top_models):
    scores = all_scores[model_name]
    ax3.hist(scores, bins=50, alpha=0.5, density=True, label=model_name)

ax3.set_xlabel('Anomaly Score')
ax3.set_ylabel('Density')
ax3.set_title('Anomaly Score Distributions (Top 5 Models)')
ax3.legend()

# Plot 4: Correlation Heatmap of Model Scores
ax4 = axes[1, 1]
if len(successful_scores) >= 3:
    score_df = pd.DataFrame(scores_array, columns=successful_models[:len(successful_scores)])
    corr_matrix = score_df.corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0.5, ax=ax4, square=True)
    ax4.set_title('Model Score Correlations')

plt.tight_layout()
plt.savefig(f'{results_dir}model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Anomaly Visualization using t-SNE
print("\nCreating t-SNE visualization...")
if X.shape[0] > 10000:
    # Sample for t-SNE if dataset is large
    print("  - Sampling 10,000 points for t-SNE...")
    sample_idx = np.random.choice(X.shape[0], 10000, replace=False)
    X_sample = X[sample_idx]
    sample_predictions = {name: result['predictions'][sample_idx] 
                         for name, result in all_results.items() 
                         if 'predictions' in result}
else:
    X_sample = X
    sample_predictions = {name: result['predictions'] 
                         for name, result in all_results.items() 
                         if 'predictions' in result}

# Apply t-SNE
print("  - Running t-SNE (this may take a few minutes)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_jobs=-1)
X_tsne = tsne.fit_transform(X_sample[:5000])  # Limit for performance

# Create visualization for top 4 models
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, (model_name, predictions) in enumerate(list(sample_predictions.items())[:4]):
    ax = axes[idx]
    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                        c=predictions[:5000], cmap='coolwarm', 
                        alpha=0.6, s=10)
    ax.set_title(f'Anomalies Detected by {model_name}')
    ax.set_xlabel('t-SNE Component 1')
    ax.set_ylabel('t-SNE Component 2')
    plt.colorbar(scatter, ax=ax)

plt.tight_layout()
plt.savefig(f'{results_dir}tsne_anomalies.png', dpi=300, bbox_inches='tight')
plt.show()

# =====================================================================
# STEP 6: DETAILED OUTLIER ANALYSIS
# =====================================================================
print("\n[STEP 6] Analyzing Detected Outliers...")

# Use ensemble average for final analysis
if 'ensemble_results' in locals():
    final_predictions = ensemble_results['Average']['predictions']
    final_scores = ensemble_results['Average']['scores']
else:
    # Use best single model
    best_model = 'IForest'
    final_predictions = all_results[best_model]['predictions']
    final_scores = all_results[best_model]['scores']

# Get outlier indices
outlier_indices = np.where(final_predictions == 1)[0]
normal_indices = np.where(final_predictions == 0)[0]

# Compare feature statistics
feature_comparison = pd.DataFrame()
feature_names = df_clean.columns[df_clean.columns.isin(feature_cols)]

for feature in feature_names[:15]:  # Top 15 features
    outlier_values = df_clean.iloc[outlier_indices][feature]
    normal_values = df_clean.iloc[normal_indices][feature]
    
    feature_comparison = pd.concat([feature_comparison, pd.DataFrame({
        'Feature': [feature],
        'Normal_Mean': [normal_values.mean()],
        'Outlier_Mean': [outlier_values.mean()],
        'Difference_%': [(outlier_values.mean() - normal_values.mean()) / 
                        (normal_values.mean() + 1e-10) * 100],
        'Normal_Std': [normal_values.std()],
        'Outlier_Std': [outlier_values.std()]
    })], ignore_index=True)

# Sort by absolute difference
feature_comparison['Abs_Difference_%'] = abs(feature_comparison['Difference_%'])
feature_comparison = feature_comparison.sort_values('Abs_Difference_%', ascending=False)

# Visualize feature differences
plt.figure(figsize=(12, 8))
top_features = feature_comparison.head(10)
x = range(len(top_features))
plt.bar(x, top_features['Difference_%'], color=['red' if d > 0 else 'blue' 
                                                for d in top_features['Difference_%']])
plt.xticks(x, top_features['Feature'], rotation=45, ha='right')
plt.ylabel('Percentage Difference (%)')
plt.title('Top 10 Features Distinguishing Outliers from Normal')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.savefig(f'{results_dir}feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# =====================================================================
# STEP 7: SAVE RESULTS AND MODELS
# =====================================================================
print("\n[STEP 7] Saving Results and Models...")

# Save the best models
best_models_to_save = ['IForest', 'ECOD', 'AutoEncoder']
for model_name in best_models_to_save:
    if model_name in all_results and 'model' in all_results[model_name]:
        model_path = f'{results_dir}{model_name}_model.pkl'
        joblib.dump(all_results[model_name]['model'], model_path)
        print(f"✓ Saved {model_name} model")

# Save preprocessing pipeline info
pipeline_info = {
    'contamination': contamination,
    'n_samples': len(X),
    'n_features': X.shape[1],
    'feature_names': feature_cols,
    'pca_components': X_reduced.shape[1] if 'X_reduced' in locals() else None,
    'models_trained': list(all_results.keys()),
    'ensemble_models': successful_models if 'successful_models' in locals() else None
}

joblib.dump(pipeline_info, f'{results_dir}pipeline_info.pkl')

# Create comprehensive report
report = []
report.append("# Advanced Unsupervised Anomaly Detection Report\n")
report.append(f"**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
report.append("---\n\n")

report.append("## 1. Dataset Summary\n")
report.append(f"- Original samples: {original_size:,}\n")
report.append(f"- Samples after cleaning: {len(X):,}\n")
report.append(f"- Features used: {X.shape[1]}\n")
report.append(f"- Contamination rate: {contamination*100}%\n\n")

report.append("## 2. Model Performance Summary\n\n")
report.append("| Model | Training Time | Outliers Detected | Outlier % | Status |\n")
report.append("|-------|--------------|-------------------|-----------|--------|\n")

for name, result in all_results.items():
    if 'error' not in result:
        report.append(f"| {name} | {result['train_time']:.2f}s | "
                     f"{result['n_outliers']:,} | {result['outlier_percentage']:.2f}% | ✓ |\n")
    else:
        report.append(f"| {name} | - | - | - | ✗ |\n")

report.append("\n## 3. Top Discriminating Features\n\n")
report.append("| Feature | Normal Mean | Outlier Mean | Difference % |\n")
report.append("|---------|-------------|--------------|-------------|\n")
for _, row in feature_comparison.head(10).iterrows():
    report.append(f"| {row['Feature']} | {row['Normal_Mean']:.3f} | "
                 f"{row['Outlier_Mean']:.3f} | {row['Difference_%']:.1f}% |\n")

report.append("\n## 4. Ensemble Results\n\n")
if 'ensemble_results' in locals():
    for name, data in ensemble_results.items():
        report.append(f"- **{name} Ensemble**: {data['n_outliers']:,} outliers "
                     f"({data['n_outliers']/len(X)*100:.2f}%)\n")
        report.append(f"  - Models used: {', '.join(data['models_used'])}\n")

report.append("\n## 5. Key Findings\n\n")
report.append("1. **Model Agreement**: ")
if 'corr_matrix' in locals():
    avg_corr = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean()
    report.append(f"Average correlation between models: {avg_corr:.3f}\n")

report.append("2. **Outlier Characteristics**: Based on feature analysis, outliers show:\n")
top_3_features = feature_comparison.head(3)
for _, row in top_3_features.iterrows():
    direction = "higher" if row['Difference_%'] > 0 else "lower"
    report.append(f"   - {abs(row['Difference_%']):.0f}% {direction} {row['Feature']}\n")

report.append("\n## 6. Recommendations\n\n")
report.append("- Use ensemble methods for production deployment\n")
report.append("- Monitor high-difference features for real-time detection\n")
report.append("- Consider retraining with updated contamination rates\n")
report.append("- Validate detected anomalies with domain experts\n")

# Save report
with open(f'{results_dir}anomaly_detection_report.md', 'w') as f:
    f.writelines(report)

print(f"\n✓ Report saved to: {results_dir}anomaly_detection_report.md")
print(f"✓ Models saved to: {results_dir}")
print(f"✓ Visualizations saved to: {results_dir}")

# =====================================================================
# FINAL SUMMARY
# =====================================================================
print("\n" + "="*80)
print("ANOMALY DETECTION PIPELINE COMPLETED")
print("="*80)
print(f"Models trained: {len([r for r in all_results.values() if 'error' not in r])}/{len(all_results)}")
print(f"Best single model: {min(all_results.items(), key=lambda x: abs(x[1].get('outlier_percentage', 100) - contamination*100) if 'outlier_percentage' in x[1] else 100)[0]}")
print(f"Ensemble models created: {len(ensemble_results) if 'ensemble_results' in locals() else 0}")
print(f"Total outliers detected (ensemble): {ensemble_results['Average']['n_outliers'] if 'ensemble_results' in locals() else 'N/A'}")
print("="*80)

ADVANCED UNSUPERVISED ANOMALY DETECTION PIPELINE

[STEP 1] Loading preprocessed data...


KeyboardInterrupt: 